# 4. Clustering dei Giocatori

**Obiettivo:** Raggruppare i giocatori in cluster omogenei basati sul loro stile di gioco, utilizzando le statistiche normalizzate e avanzate.

**Fasi:**
1.  **Caricamento e Selezione Dati:** Caricamento del dataset arricchito e filtraggio per mantenere solo la stagione più recente di ogni giocatore, al fine di analizzare il loro profilo attuale.
2.  **Selezione e Preparazione delle Feature:** Scelta delle statistiche più rappresentative dello stile di gioco e loro standardizzazione per il modello di clustering.
3.  **Determinazione di 'k' (Numero di Cluster):** Utilizzo dell'Elbow Method per identificare il numero ottimale di cluster da creare.
4.  **Addestramento e Valutazione:** Addestramento del modello K-Means con il 'k' ottimale e valutazione della qualità dei cluster tramite il Silhouette Score.
5.  **Salvataggio dei Risultati:** Salvataggio del DataFrame con l'ID del cluster assegnato a ogni giocatore.

In [ ]:
import os
import sys
import matplotlib.pyplot as plt
import seaborn as sns
from pyspark.sql.functions import col, max 
from pyspark.ml.clustering import KMeans

# Aggiunge la root del progetto al sys.path
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

# Importazione delle utility e delle funzioni di clustering
from src.utils.helpers import get_spark_session, save_dataframe
from src.clustering.models import prepare_features_for_clustering, train_kmeans_model, assign_clusters, evaluate_clustering
from src.config.spark_config import SPARK_CONFIG

# Inizializzazione della sessione Spark
spark = get_spark_session(
    app_name="NBA_Player_Clustering",
    driver_memory=SPARK_CONFIG["driver_memory"]
)

### Fase 1: Caricamento Dati e Selezione dell'Ultima Stagione

In [ ]:
# Carica il dataset con le metriche avanzate
adv_metrics_path = "../data/processed/players_advanced_metrics.parquet"
df_input = spark.read.parquet(adv_metrics_path)

# Per analizzare lo stile di gioco attuale, filtriamo per l'ultima stagione di ogni giocatore
df_latest_season = df_input.groupBy("player").agg(max(col("season")).alias("latest_year"))
df_for_clustering_input = df_input.join(
    df_latest_season,
    (df_input.player == df_latest_season.player) & (df_input.season == df_latest_season.latest_year),
    "inner"
).select(df_input["*"])

# Definiamo le feature che meglio descrivono lo stile di gioco di un giocatore
feature_cols = [
    'pts_per_36_min', 'trb_per_36_min', 'ast_per_36_min', 
    'stl_per_36_min', 'blk_per_36_min', 'tov_per_36_min',
    'ts_pct_calc'  # L'efficienza è una feature cruciale
]

# Prepara il DataFrame finale per il clustering, rimuovendo valori nulli
df_for_clustering = df_for_clustering_input.select(["player", "season"] + feature_cols).na.drop()
print(f"Numero di giocatori idonei per il clustering: {df_for_clustering.count()}")

### Fase 2: Preparazione e Scaling delle Feature

Le feature vengono assemblate in un unico vettore e poi scalate (standardizzate) per garantire che nessuna singola statistica domini l'algoritmo a causa della sua scala (es. i punti sono numericamente maggiori dei blocchi).

In [ ]:
# Applica la funzione per assemblare e scalare le feature
df_prepared = prepare_features_for_clustering(df_for_clustering, feature_cols)
df_prepared.select("player", "features_scaled").show(5, truncate=False)

### Fase 3: Determinazione del Numero Ottimale di Cluster (Elbow Method)

Addestriamo il modello K-Means per diversi valori di 'k' e plottiamo il costo (WSSSE - Within Set Sum of Squared Errors). Il punto in cui la curva si "appiattisce" (il "gomito") suggerisce un buon compromesso per 'k'.

In [ ]:
# Calcolo del costo (WSSSE) per un range di valori di k
cost = []
k_range = range(2, 15)
print("Calcolo del costo WSSSE per diversi k...")

for k in k_range:
    kmeans = KMeans(featuresCol='features_scaled', k=k, seed=42)
    model = kmeans.fit(df_prepared)
    cost.append(model.summary.trainingCost)

# Visualizzazione del grafico dell'Elbow Method
plt.figure(figsize=(12, 6))
plt.plot(k_range, cost, 'bx-')
plt.xlabel('k (Numero di Cluster)')
plt.ylabel('Costo (WSSSE)')
plt.title('Elbow Method per la Scelta Ottimale di k')
plt.xticks(k_range)
plt.grid(True)
plt.show()

**Analisi del Grafico:**
Il grafico mostra un "gomito" evidente intorno a `k=6`. Aggiungere più cluster oltre questo punto porta a benefici decrescenti. Scegliamo quindi `k=6` perché offre un buon equilibrio tra numero di profili e loro interpretabilità.

### Fase 4: Addestramento del Modello Finale e Valutazione

In [ ]:
# Scelta del valore di k basata sull'analisi del gomito
k_optimal = 6
print(f"Addestramento del modello K-Means finale con k={k_optimal}...")

# Addestramento e assegnazione dei cluster
kmeans_model = train_kmeans_model(df_prepared, k=k_optimal)
predictions_df = assign_clusters(kmeans_model, df_prepared)

# Visualizzazione dei risultati per alcuni giocatori
print("Esempi di giocatori con cluster assegnato:")
predictions_df.select("player", "season", "cluster_id").show(10)

# Valutazione della coesione e separazione dei cluster con il Silhouette Score
silhouette_score = evaluate_clustering(predictions_df)
print(f"Punteggio Silhouette (validità del clustering): {silhouette_score:.3f}")

### Fase 5: Salvataggio dei Risultati del Clustering

In [ ]:
# Selezione delle colonne rilevanti per il salvataggio (escludendo i vettori di MLlib)
columns_to_save = ["player", "season"] + feature_cols + ["cluster_id"]
df_to_save = predictions_df.select(columns_to_save)

# Salvataggio dei risultati in formato Parquet
output_path = "../data/processed/players_clustered.parquet"
save_dataframe(df_to_save, output_path)

print(f"Risultati del clustering salvati con successo in '{output_path}'.")

In [ ]:
# Termina la sessione Spark
spark.stop()